In [16]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
from src.pipeline_functions import *

In [14]:
with open("../src/data/news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())

In [21]:
days = 21
df_tweets = load_tweets("../home.db", days) # load tweets
# convert columns to avoid merge problems
for col in ["in_reply_to_status_id"]:
    df_tweets[col] = df_tweets[col].fillna(0).astype(np.int64)

df_tweets = find_news(df_tweets, news_domains) # add news column
df_tweets = news_in_qt_rt(df_tweets) # find news in reweets and reply-to

seen_tweets = pd.read_csv("../src/data/seen.csv")
seen_tweets.drop_duplicates(inplace=True)

df_tweets = df_tweets[~df_tweets["id"].isin(seen_tweets["tweet_id"].tolist())] #filter out seen tweets

df_tweets = df_tweets[df_tweets["lang"] == "en"] # take only english lang tweets

# filter out tweets with news links
to_custom_news_feed = (
    df_tweets[df_tweets["all_news"] == 0]
    .sample(frac=1)
    .reset_index(drop=True)[:1000]
)
to_custom_news_feed = drop_contains(to_custom_news_feed, column_name="full_text", str_list = ["breaking:", "🍿", "New Yorker", "racially", "lived experience"])
to_custom_news_feed = drop_contains(to_custom_news_feed, column_name="full_text", str_list = ["GOP"], lower=False)

df = to_custom_news_feed[["id", "user"]]

FileNotFoundError: [Errno 2] No such file or directory: 'src/data/batch_to_add.csv'

In [22]:
to_custom_news_feed.head()

,id,user,full_text,created_at,lang,quoted_status,in_reply_to_status_id,contains_news,all_news,qt_news,rt_news
0,1344708879968473088,48562581,This is fucked https://twitter.com/justinbolda...,2020-12-31T18:15:55+00:00,en,NaN,0,0,0,0,0
1,1345736083552821252,2859448331,RT @nberlat: anyway. I'm a little bitter for g...,2021-01-03T14:17:39+00:00,en,NaN,0,0,0,0,0
2,1337805687699755009,1024617904426442752,It is weekend. Relaxing with #NoteAir...\n\n#W...,2020-12-12T17:05:05+00:00,en,NaN,0,0,0,0,0
3,1342533762416955393,1033612177,5 mins to assemble drone. 30 mins to untangle ...,2020-12-25T18:12:46+00:00,en,NaN,0,0,0,0,0
4,1336302195264823301,15958614,Official: AirPods Max over ear headphones comi...,2020-12-08T13:30:45+00:00,en,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1340439215788740609,1066898457678540800,Observation 6: ORNAMENTS ARE EXPENSIVE.\n\nTha...,2020-12-19T23:29:47+00:00,en,NaN,1340439210906562561,0,0,0,0
996,1344764597253726210,191666539,@michael_nielsen Yes to all parts of that!,2020-12-31T21:57:19+00:00,en,NaN,1344763281337057280,0,0,0,0
997,1330535419650379780,1218675661046538240,Wow. Can I buy this poster size? https://twitt...,2020-11-22T15:35:38+00:00,en,1.330235e+18,0,0,0,0,0
998,1344230309710147585,3448284313,RT @omarsar0: My recipe if I were starting wit...,2020-12-30T10:34:15+00:00,en,NaN,0,0,0,0,0
